In [1]:
import numpy as np
import pygame
import gymnasium as gym
from gymnasium import spaces
import math
from bond_graph import *
from bond_graph_nodes import*
from itertools import permutations
import random
import copy
from gymnasium.envs.registration import register

pygame 2.5.2 (SDL 2.30.2, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


In [52]:
seed = None
MAX_PARAM_VAL = 10
num_node_types = 6
max_nodes = 8

add_node_space = spaces.Discrete(num_node_types-3, start=3, seed=seed) # node additions correspond to choosing what type you want, don't include the NONE type for adding
add_edge_space = spaces.MultiDiscrete([max_nodes, max_nodes, 2], seed=seed) # edge additions sample space

action_space = spaces.Dict(
    {
        'node_or_bond': spaces.Discrete(2, start=0, seed=seed),
        'node_param': spaces.Discrete(MAX_PARAM_VAL, start=1, seed=seed),
        "node_type": add_node_space,
        "bond": add_edge_space,
    }
)

print(add_node_space.sample())

# print("Action Space: ", action_space)

flattened_action_space = spaces.utils.flatten_space(action_space)
print("Flattened Action Space: ", flattened_action_space)
print(flattened_action_space.sample())

# obs = action_space.sample()
# flat_obs = spaces.utils.flatten(action_space, obs)
# print("Obs: ", obs)
# print("Flat Obs: ", flat_obs)
# print(flat_obs.shape)

# unflattened_obs = spaces.utils.unflatten(action_space, flat_obs)
# print(unflattened_obs)


3
Flattened Action Space:  Box(0, 1, (33,), int64)
[0 1 1 1 1 0 0 0 1 0 0 0 1 1 0 0 1 0 1 0 1 0 0 0 1 0 1 0 1 1 0 0 1]


In [11]:
# from gymnasium.spaces import Dict, Discrete, Box
space = spaces.Dict(
    {
        "position": spaces.Discrete(2), 
        "velocity": spaces.Box(0, 1, shape=(2, 2))
    }
) 
obs = space.sample()
print(obs)

flat_space = spaces.utils.flatten_space(space) 
print(flat_space.shape)
flat_obs = spaces.utils.flatten(space, obs)
print(flat_obs)


OrderedDict({'position': 0, 'velocity': array([[0.1838431, 0.8598087],
       [0.660587 , 0.7158184]], dtype=float32)})
(6,)
[1.         0.         0.18384311 0.85980868 0.66058701 0.71581841]


In [47]:
adjacency_matrix_space = spaces.Box(low=0, high=1, shape=(max_nodes, max_nodes), dtype=np.int64) # represents the flow-causal adacency matrix
node_type_space = spaces.MultiDiscrete([3]*10, seed=seed) # look at up to the number of max_nodes
node_parameter_space = spaces.MultiDiscrete([5]*10, seed=seed)

## Dictionary
observation_space = spaces.Dict(
    {
        "adjacency_matrix_space": adjacency_matrix_space,
        "node_type_space": node_type_space,
        "node_param_space": node_parameter_space
    }
)

print(observation_space)
print(observation_space.sample())
print()

## Tuple
# observation_space = spaces.Tuple((adjacency_matrix_space, node_type_space, node_parameter_space))
# print(observation_space)
# print(observation_space.sample())
# print()

n_observations = env.flattened_observation_space.shape[0]

Dict('adjacency_matrix_space': Box(0, 1, (8, 8), int64), 'node_param_space': MultiDiscrete([5 5 5 5 5 5 5 5 5 5]), 'node_type_space': MultiDiscrete([3 3 3 3 3 3 3 3 3 3]))
OrderedDict({'adjacency_matrix_space': array([[0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 1],
       [1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 1, 1, 1],
       [1, 1, 1, 0, 0, 1, 0, 0]]), 'node_param_space': array([4, 3, 4, 2, 0, 1, 2, 0, 2, 1]), 'node_type_space': array([1, 0, 1, 0, 1, 0, 2, 2, 2, 1])})

Box(0, 1, (144,), int64)
[1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 0 0 1 0 0
 1 1 0 0 1 0 1 0 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 0 0 1 0 0 1 0
 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 0 1 0 0 1 0 0 0 1
 1 0 1 0 0 1 1 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 0 1 1 0 1 0 0 1 1]


### Test manual mapping of integer to composite action space

In [82]:
action_space_indices = []
MIN_PARAM_VAL = 1
MAX_PARAM_VAL = 2500
num_node_types = 3
for node_or_bond in range(2):
    for node_param in range(MIN_PARAM_VAL, MAX_PARAM_VAL):
        for node_type in range(3, 3+num_node_types):
            for bond1 in range(max_nodes):
                for bond2 in range(max_nodes):
                    action_index = [node_or_bond, node_param, node_type, bond1, bond2]
                    action_space_indices.append(action_index)

print(action_space_indices[1])
print(len(action_space_indices))


[0, 1, 3, 0, 1]
959616
